# Y~G ACGT Probabilities 

> 

In [2]:
import numpy as np
import pandas as pd

from EnvDL.core import ensure_dir_path_exists 
from EnvDL.dlfn import g2fc_datawrapper, BigDataset, plDNN_general
from EnvDL.dlfn import ResNet2d, BasicBlock2d
from EnvDL.dlfn import LSUV_

import torch
import torch.nn.functional as F # F.mse_loss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger

In [3]:
# Run settings: 
max_epoch = 2
batch_size = 24

In [4]:
use_gpu_num = 0

device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [5]:
cache_path = '../nbs_artifacts/02.20_g2fc_G_ACGT_FCN/'
ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [6]:
X = g2fc_datawrapper()
X.set_split()
X.load_all(name_list = ['obs_geno_lookup', 'YMat', 'ACGT',], store=True) 

X.calc_cs('YMat', version = 'np', filter = 'val:train')
X.calc_cs('ACGT',                 filter = 'val:train', filter_lookup= 'obs_geno_lookup')

Loading and storing default `phno`.


In [7]:
training_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs  = X.get('val:train',       ops_string='   asarray from_numpy      '),
    lookup_geno = X.get('obs_geno_lookup', ops_string='   asarray from_numpy      '),
    y =           X.get('YMat',            ops_string='cs asarray from_numpy float')[:, None],
    G =           X.get('ACGT',            ops_string='           from_numpy float'),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = True
)


validation_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs = X.get('val:test',         ops_string='   asarray from_numpy             '),
    lookup_geno = X.get('obs_geno_lookup', ops_string='   asarray from_numpy             '),
    y =          X.get('YMat',             ops_string='cs asarray from_numpy float')[:, None],
    G =          X.get('ACGT',             ops_string='           from_numpy float'),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = False
)

## Test Models

In [9]:
def Linear_block(in_size, out_size, drop_pr):
            block = nn.Sequential(
                nn.Linear(in_size, out_size),
                nn.ReLU(),
                nn.Dropout(drop_pr)
            )
            return(block) 

In [23]:
# A quirk of this is that to get only a single layer the length of the input tensor must be passed in. for 2+ I'll figure it out.
class NeuralNetwork(nn.Module):
    def __init__(self, parameterization):
        super(NeuralNetwork, self).__init__()            
        module_list = []

        max_layer = parameterization['num_layers']
        for i in range(max_layer):
            if i  == 0:
                name_in = f"in_{i+1}_of_{max_layer}"
            else:
                name_in = f"out_{i}_of_{max_layer}"
            name_out = f"out_{i+1}_of_{max_layer}"
            name_drop= f"drop_{i+1}_of_{max_layer}"

            if i == 0:
                module_list += [nn.Flatten()]
            

            module_list += [
                Linear_block(
                    in_size  = parameterization[name_in], 
                    out_size = parameterization[name_out], 
                    drop_pr  = parameterization[name_drop])]
            
            if (i+1) == max_layer:
                module_list += [nn.Linear(parameterization[name_out], 1)]
                
        self.x_network = nn.ModuleList(module_list)
        
    def forward(self, x):
        
        for mod in self.x_network:
            if mod == self.x_network[-1]:
                out = x # get the penultimate layer's outputs for later
            x = mod(x)
        
        pred = x
        return pred#, out

# model = NeuralNetwork(
#     parameterization = parameterization).to(device)

# model(next(iter(training_dataloader))[0][0:5])

In [24]:
[e.shape for e in next(iter(training_dataloader))]

[torch.Size([24, 1]), torch.Size([24, 4, 125891])]

In [25]:
# convenince wrapper to fill in for R's seq or x:y notation
def linrange(start, stop):
    import numpy as np
    diff = start - stop
    res = np.linspace(start, stop, abs(diff)+1).astype(int)
    return res 

### Tiny 2 layer block model

In [28]:
layer_sizes = [256]
layer_drops = [0.1 for e in layer_sizes]

num_layers = len(layer_sizes)

params = {
    'num_layers':num_layers,
    f"in_1_of_{num_layers}": (4 * 125891)
}

for i in range(num_layers):
    params[f"out_{ i + 1}_of_{num_layers}"] = layer_sizes[i]
    params[f"drop_{ i + 1}_of_{num_layers}"] = layer_drops[i]
        

In [29]:
model = NeuralNetwork(parameterization = params).to(device)
# model

NeuralNetwork(
  (x_network): ModuleList(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Sequential(
      (0): Linear(in_features=503564, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
    )
    (2): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [30]:
# model(next(iter(training_dataloader))[1])


In [31]:
LSUV_(model, data = next(iter(training_dataloader))[1] )

Applying orthogonal init (zero init if dim < 2) to params in 2 module(s).
Applying LSUV to 2 module(s) (up to 10 iters per module):
Module  0 after  2 itr(s) | Mean:  0.033 | Std: 1.000 | <class 'torch.nn.modules.linear.Linear'>
Module  1 after  3 itr(s) | Mean:  1.009 | Std: 0.995 | <class 'torch.nn.modules.linear.Linear'>


In [32]:
DNNG = plDNN_general(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-tiny")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | mod  | NeuralNetwork | 128 M 
---------------------------------------
128 M     Trainable params
0         Non-trainable params
128 M     Total params
515.652   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


### Deeper linear block model

In [33]:
            # [2048, 1024, 512, 256, 128, 64, 32, 16]
layer_sizes = [int(2**i) for i in linrange(start = 11, stop = 4)]
layer_drops = [0.1 for e in layer_sizes]

num_layers = len(layer_sizes)

params = {
    'num_layers':num_layers,
    f"in_1_of_{num_layers}": (4 * 125891)
}

for i in range(num_layers):
    params[f"out_{ i + 1}_of_{num_layers}"] = layer_sizes[i]
    params[f"drop_{ i + 1}_of_{num_layers}"] = layer_drops[i]
        

In [34]:
model = NeuralNetwork(parameterization = params).to(device)
# model
# model(next(iter(training_dataloader))[1])

LSUV_(model, data = next(iter(training_dataloader))[1] )

Applying orthogonal init (zero init if dim < 2) to params in 9 module(s).
Applying LSUV to 9 module(s) (up to 10 iters per module):
Module  0 after  2 itr(s) | Mean: -0.025 | Std: 1.000 | <class 'torch.nn.modules.linear.Linear'>
Module  1 after  2 itr(s) | Mean: -0.020 | Std: 1.002 | <class 'torch.nn.modules.linear.Linear'>
Module  2 after  2 itr(s) | Mean:  0.009 | Std: 0.994 | <class 'torch.nn.modules.linear.Linear'>
Module  3 after  2 itr(s) | Mean:  0.004 | Std: 1.013 | <class 'torch.nn.modules.linear.Linear'>
Module  4 after  2 itr(s) | Mean: -0.074 | Std: 0.996 | <class 'torch.nn.modules.linear.Linear'>
Module  5 after  2 itr(s) | Mean:  0.030 | Std: 0.995 | <class 'torch.nn.modules.linear.Linear'>
Module  6 after  2 itr(s) | Mean:  0.058 | Std: 1.014 | <class 'torch.nn.modules.linear.Linear'>
Module  7 after  2 itr(s) | Mean:  0.396 | Std: 0.959 | <class 'torch.nn.modules.linear.Linear'>
Module  8 after  3 itr(s) | Mean: -0.557 | Std: 0.923 | <class 'torch.nn.modules.linear.Line

In [35]:
DNNG = plDNN_general(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-fcn")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | mod  | NeuralNetwork | 1.0 B 
---------------------------------------
1.0 B     Trainable params
0         Non-trainable params
1.0 B     Total params
4,136.397 Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [ ]:
# torch.save(DNNG.mod, cache_path+'g-acgt-fcn'+'.pt')

In [ ]:
# os._exit(00)